# Modelling - Bettina Pölzleitner (Lead) and Jonas Gstöttenmayr (Assistant)
Finding the best combination of predictor and dataset features will be determined and validated.

add to say model_stats/ml/dl is exploratory


In [2]:
import torch
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pathlib import Path


torch.cuda.is_available()

True

In [3]:
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

In [4]:
data_path = Path("../data/processed_data")
parquet_files = list(data_path.glob("*.parquet"))
# dfs = {f.stem: pl.read_parquet(f) for f in parquet_files}

In [5]:
dicct = {"train": {}, "val": {},"test": {}, "future": {}, "train_future": {}, "val_future": {},  "test_future": {}}
mapping = {"fourier+trend+holidays": "fth", "fourier": "fou", "holidays": "hol", "none": "non", "trend": "tre"}
for f in parquet_files:
    name = f.stem
    split = name.split("_", 1)
    if split[-1] in dicct.keys():
        dicct[split[-1]][mapping[split[0]]] = pl.read_parquet(f)

In [6]:
dicct["train"]["non"]

ds,unique_id,y
date,str,f64
2020-07-01,"""Bubble tea""",2012.0
2020-07-02,"""Bubble tea""",2085.0
2020-07-03,"""Bubble tea""",2204.0
2020-07-04,"""Bubble tea""",2119.0
2020-07-05,"""Bubble tea""",2176.0
…,…,…
2025-09-26,"""Tea""",1206.0
2025-09-27,"""Tea""",1074.0
2025-09-28,"""Tea""",1222.0


## Statistical Models

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoRegressive, SeasonalExponentialSmoothingOptimized, HoltWinters, 

c:\Users\jojog\Documents\School\Semester_3\TIS3\cafe_upscaling_models\.venv\Lib\site-packages\statsforecast\models.py:3806: SyntaxWarning: invalid escape sequence '\h'
  """CrostonClassic model.
c:\Users\jojog\Documents\School\Semester_3\TIS3\cafe_upscaling_models\.venv\Lib\site-packages\statsforecast\models.py:3984: SyntaxWarning: invalid escape sequence '\h'
  """CrostonOptimized model.
c:\Users\jojog\Documents\School\Semester_3\TIS3\cafe_upscaling_models\.venv\Lib\site-packages\statsforecast\models.py:4132: SyntaxWarning: invalid escape sequence '\h'
  """CrostonSBA model.
c:\Users\jojog\Documents\School\Semester_3\TIS3\cafe_upscaling_models\.venv\Lib\site-packages\statsforecast\models.py:4468: SyntaxWarning: invalid escape sequence '\e'
  """TSB model.


In [23]:
def train_stats_model(dataset):
    season_length = 7 # Monthly data
    horizon = int(len(dicct["test"]["non"])/10) # number of predictions

    models = [AutoARIMA(season_length=season_length),
            AutoRegressive(lags=[14], include_mean=True),
            SeasonalExponentialSmoothingOptimized(season_length=season_length),
            HoltWinters(season_length=season_length, error_type="A", alias="Add")]
    
    sf = StatsForecast(models=models, freq='D', n_jobs = -1)

    sf.fit(df=dicct["train"][dataset])
    
    preditions = sf.predict(h = horizon)
    return preditions

In [24]:
train_stats_model("non")

,unique_id,ds,AutoARIMA,AutoRegressive,SeasESOpt,Add
0,"""Bubble tea""",2025-10-01,1235.053173,1603.159410,1201.817308,1178.244696
1,"""Bubble tea""",2025-10-02,1285.240607,1514.111142,1189.601772,1216.963902
2,"""Bubble tea""",2025-10-03,1441.943523,1651.168042,1475.429401,1343.298368
3,"""Bubble tea""",2025-10-04,1467.632445,1696.079342,1534.828116,1520.046579
4,"""Bubble tea""",2025-10-05,1345.414798,1650.393709,1404.869409,1516.852784
...,...,...,...,...,...,...
305,"""Tea""",2025-10-27,1214.627571,1436.114396,1240.654246,1204.376328
306,"""Tea""",2025-10-28,1216.719091,1442.204258,1259.278983,1222.421475
307,"""Tea""",2025-10-29,1214.777405,1530.822126,1240.372584,1238.689590
308,"""Tea""",2025-10-30,1216.580007,1545.850469,1198.768587,1216.639854


## Machine Learning Models

In [25]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from window_ops.expanding import expanding_mean

In [26]:
# Import the necessary models from various libraries

# LGBMRegressor: A gradient boosting framework that uses tree-based learning algorithms from the LightGBM library
from lightgbm import LGBMRegressor

# XGBRegressor: A gradient boosting regressor model from the XGBoost library
from xgboost import XGBRegressor

# LinearRegression: A simple linear regression model from the scikit-learn library
from sklearn.linear_model import LinearRegression

In [27]:
# Instantiate the MLForecast object
mlf = MLForecast(
    models=[LGBMRegressor(), XGBRegressor(), LinearRegression()],  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
    freq='D',  # Frequency of the data - 'D' for daily frequency
    lags=list(range(1, 7)),  # Specific lags to use as regressors: 1 to 6 days
    lag_transforms = {
        1:  [expanding_mean],  # Apply expanding mean transformation to the lag of 1 day
    },
    date_features=['ds'],  # Date features to use as regressors
)

In [30]:
dicct["test"]["non"]["ds"]

0      2025-11-01
1      2025-11-02
2      2025-11-03
3      2025-11-04
4      2025-11-05
          ...    
305    2025-11-27
306    2025-11-28
307    2025-11-29
308    2025-11-30
309    2025-12-01
Name: ds, Length: 310, dtype: object

In [35]:
# Jonas, pls do thisssss
# I cant do this aaaaaaaaaahhhhhhhh

for split, parts in dicct.items():
    for key, df in parts.items():
        dicct[split][key] = df.with_columns(
            pl.col("ds").str.to_datetime()
        ).sort("ds")

AttributeError: 'DataFrame' object has no attribute 'with_columns'

In [36]:
train_non_pd = dicct["train"]["non"].to_pandas()
mlf.fit(train_non_pd, prediction_intervals=PredictionIntervals(n_windows=28))

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

## Deep Learning Models

## Please read (to Jonas)

I am sorry for not finishing this but I am so done with the types mismatch and you are way better at this than I am. I owe you for this. I am so sorry. Also I should hydrate. 

In [38]:
import logging

import pandas as pd
from utilsforecast.plotting import plot_series

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS

In [40]:
y = dicct["test"]["non"]
y_val = dicct["val"]["non"]
# Fit and predict with NBEATS and NHITS models
horizon = len(y)
models = [NBEATS(input_size=2 * horizon, h=horizon, max_steps=100, enable_progress_bar=False),
          NHITS(input_size=2 * horizon, h=horizon, max_steps=100, enable_progress_bar=False)]
nf = NeuralForecast(models=models, freq='ME')
nf.fit(df=y)
Y_hat_df = nf.predict()

# Plot predictions
plot_series(y, y_val)

Seed set to 1
Seed set to 1


ValueError: The time column ('ds') should have either timestamps or integers, got 'object'.

In [ ]:
sf.plot(Y_df, fcst_df, max_insample_length=28 * 3)

In [ ]:
sf.plot(Y_df, fcst_df, max_insample_length=28 * 3,
        models=['CrostonOptimized', 'AutoNHITS', 'SeasonalNaive', 'LGBMRegressor'])

In [ ]:
   sf = StatsForecast(models=models, freq='D', n_jobs = -1)